# Content Scraper - General Based on Body

### Agenda
1. Mengambil seluruh data yang ada di dalam link yang diberikan
2. Memilih data yang akan digunakan
3. Menyimpan data ke dalam dokumen csv/pickle

### Load library yang dibutuhkan

In [1]:
# Library untuk handle http request
import urllib2
import requests as rq

# Library untuk mengolah CSV
import csv

# Library untuk mengolah text hasil scraping 
from bs4 import BeautifulSoup

# Library untuk mengolah data
import pandas as pd

# Module untuk setting encoding
import sys

# hash untuk filename
import hashlib
import time

# Regex
import re


### Mengambil seluruh data dari link yang diberikan

In [2]:
# for Python 2: use print only as a function
from __future__ import print_function

#### Config

- File berisi URL yang aka digrab
- Tag Identifier, tag yang untuk mengidentifikasi letak konten
- Path directory output

In [3]:
name = 'Style_and_Fashion_IAB18'

# doc = './data/links/Sport_IAB17.csv'
doc = './data/links/'+ name + '.csv'

identifier = {'class': 'article-content'}
path = 'output/' + name + '/'

#### Function: grab_urls
Untuk mengambil seluruh data (links) dari dokument yang diberikan
- **input**: document yang berisi link
- **output**: link dengan format data list

In [4]:
def grab_urls(doc):
    urls = []
    with open(doc, 'rb') as csvfile:
        links = csv.reader(csvfile)
        for link in links:
            urls.append(link)
    return urls

#### Function: build_content
Untuk mengekstrak konten yg hasil proses beautifulsoup
- **input**: konten soup
- **output**: konten yang sudah bersih dari tag html

In [5]:
# build content
def build_content(soup, attr):
    contents = []
    title = ""
    textblock = ""
    try:
        title = soup.find('h1')
        if title == "":
            title == soup.find('h2')
        else:
            title == soup.find('h3')
            
        content = soup.find('body')
        pages = content.findAll(['p'])
        for page in pages:
            contents.append(page.getText())
        
        print(title)
        return contents
    except:
        return False

#### Funtion name: make_hash
- output: hash 

In [6]:
def make_hash():
    hash = hashlib.sha1()
    hash.update(str(time.time()))
    return str(hash.hexdigest())

** Function name: clean_data **
- input: raw text from internet
- output: removed unnecesary char and white space

In [7]:
def clean_data(text):
    return re.sub(r'(?m)([a-z])[\r\n]+$','\\1 ', text)

In [8]:
def make_content(urls):
    res = {}
    for idx, url in enumerate(urls):
        print("Processing {}...".format(url))
        try:
            page = rq.get(url[0])
            soup = BeautifulSoup(page.text, "html.parser")

            new_doc = path + make_hash() + ".txt"

            with open(new_doc,"wb") as site:
                content = build_content(soup, identifier)
                if content != False:
                    site.write(" ".join(content) + '\n')
                    res.update({idx : True})
                else:
                    res.update({idx : False})
        except:
            continue

#### 1. Mengambil seluruh links yang ada di dokumen

In [9]:
urls = grab_urls(doc)

#### 2. Grab the content and store it to text file

In [10]:
reload(sys)
sys.setdefaultencoding('utf-8')
make_content(urls)

#### 3. Data Cleansing

In [11]:
# import glob

In [12]:
# style_filenames = glob.glob('./output/Sport_IAB17/*.txt')

In [13]:
# for doc in style_filenames:
#     with open(doc, 'r') as f:
#         cleaned_data = clean_data(f.read())
#         f.write(" ".join(cleaned_data) + '\n')